# Лабораторная работа 3
Исследование алгоритмов классификации

### Классы

В статистике и анализе данных классом называют группу объектов или явлений, обладающих общими свойствами. Понятие класса играет важную роль в технологиях обнаружения и представление знаний. Например, среди заемщиков банка можно выделить классы добросовестных (которые не допускают просрочки) и недобросовестных (допускают). Также клиентов можно разбить на классы по уровню их активности (активный, пассивный) и т.д.

В отличие от кластеров, которые формируются в процессе кластеризации, классы должны быть определены заранее. Поэтому обучение классификаторов — это задача обучения с учителем.

Классы бывают:
- непересекающиеся -  одно наблюдение может одновременно принадлежать только одному классу;
- пересекающиеся - одно и то же наблюдение может принадлежать нескольким классам одновременно;
- нечеткими - наблюдение принадлежит к классу с некоторой степенью принадлежности (обычно степень принадлежности задается в интервале от 0 до 1).


### Классификация

Одной из важнейших задач анализа данных является классификация — отнесение объектов предметной области к заранее определённым группам, называемым классами. При этом каждому классу должны принадлежать объекты, близкие по своим свойствам. Обобщая свойства известных объектов класса на новые, отнесённые к нему объекты, можно получать знания о них.

Задача классификации решается с помощью аналитических моделей, называемых классификаторами. Классифицировать объект означает предъявить набор его признаков (обычно представленных в виде вектора) на вход модели-классификатора, которая должна присвоить ему метку или номер класса.

Необходимость использования в анализе данных большого числа разнообразных методов классификации, обусловлена тем, что решаемые с её помощью задачи могут иметь свои особенности, связанные, например, с представлением исходных данных, их количеством и качеством, что требует выбора адекватного классификатора. Поэтому выбор классификатора, соответствующего особенностям решаемой задачи анализа, является важным фактором получения правильного решения.


### Алгоритм kNN
В случае использования метода для классификации объект присваивается тому классу, который является наиболее распространённым среди k соседей данного элемента, классы которых уже известны.

Поскольку признаки, на основе которых производится классификация могут иметь различную физическую природу и, соответственно, диапазоны значений, для улучшения результатов классификации будет полезно выполнить нормализацию обучающих данных.

Для репрезентативности результатов выберем датасет отличающийся от 1,2 лабораторных работ.
Описание выбранного датасета представлено [тут](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database?resource=download).

За основу выполнения задания был выбран https://www.kaggle.com/code/shrutimechlearn/step-by-step-diabetes-classification-knn-detailed


### OSEMN Pipeline
Перед иследованием алгоритмов классификации необходимо пройти по приведенному пайплайну.

O - Obtaining our data
S - Scrubbing / Cleaning our data
E - Exploring / Visualizing our data will allow us to find patterns and trends
M - Modeling our data will give us our predictive power as a wizard
N - INterpreting our data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

diabetes_data = pd.read_csv('diabetes.csv')
diabetes_data.head()

In [ ]:
diabetes_data.describe()

In [ ]:
diabetes_data.dtypes

In [ ]:
## gives information about the data types,columns, null value counts, memory usage etc
## https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html
diabetes_data.info(verbose=True)

In [ ]:
diabetes_data_copy = diabetes_data.copy(deep = True)
diabetes_data_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = diabetes_data_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

## showing the count of Nans
print(diabetes_data_copy.isnull().sum())

In [ ]:
p = diabetes_data.hist(figsize = (20,20))

In [ ]:
# На основе построенных выше гистограм заменяем NaN медианой или средним значением в зависимости от распределения
diabetes_data_copy['Glucose'].fillna(diabetes_data_copy['Glucose'].mean(), inplace = True)
diabetes_data_copy['BloodPressure'].fillna(diabetes_data_copy['BloodPressure'].mean(), inplace = True)
diabetes_data_copy['SkinThickness'].fillna(diabetes_data_copy['SkinThickness'].median(), inplace = True)
diabetes_data_copy['Insulin'].fillna(diabetes_data_copy['Insulin'].median(), inplace = True)
diabetes_data_copy['BMI'].fillna(diabetes_data_copy['BMI'].median(), inplace = True)

In [ ]:
diabetes_data_copy.hist(figsize = (20,20))

In [ ]:
sns.pairplot(diabetes_data_copy, hue = 'Outcome')

In [ ]:
sns.heatmap(diabetes_data_copy.corr(), annot=True,cmap ='RdYlGn')

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X =  pd.DataFrame(sc_X.fit_transform(diabetes_data_copy.drop(["Outcome"],axis = 1),),
                  columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
                           'BMI', 'DiabetesPedigreeFunction', 'Age'])

In [ ]:
X.head()

In [ ]:
#X = diabetes_data.drop("Outcome",axis = 1)
y = diabetes_data_copy.Outcome

Train Test Split : To have unknown datapoints to test the data rather than testing with the same points with which the model was trained. This helps capture the model performance much better.
[Источник](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6)

In [ ]:
#importing train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=1/3,random_state=42, stratify=y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

test_scores = []
train_scores = []

for i in range(1,15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)

    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

In [ ]:
## score that comes from testing on the same datapoints that were used for training
max_train_score = max(train_scores)
train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
print('Max train score {} % and k = {}'.format(max_train_score*100,list(map(lambda x: x+1, train_scores_ind))))

In [ ]:
## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

#### Визуализируем результаты

In [ ]:
plt.figure(figsize=(12,5))
p = sns.lineplot(x=range(1,15),y=train_scores,marker='*',label='Train Score')
p = sns.lineplot(x=range(1,15),y=test_scores,marker='o',label='Test Score')

In [ ]:
# Определили по графику выше k = 11

In [ ]:
knn = KNeighborsClassifier(11)

knn.fit(X_train,y_train)
knn.score(X_test,y_test)

### Сравниv полученные результаты с помощью различных метрик оценки качества

#### ROC - AUC
[Источник](https://pythonru.com/baza-znanij/sklearn-roc-auc)

Полное название ROC — Receiver Operating Characteristic (рабочая характеристика приёмника). Впервые она была создана для использования радиолокационного обнаружения сигналов во время Второй мировой войны. США использовали ROC для повышения точности обнаружения японских самолетов с помощью радара. Поэтому ее называют рабочей характеристикой приемника.

In [ ]:
from sklearn.metrics import roc_curve
y_pred_proba = knn.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

In [ ]:
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr,tpr, label='Knn')
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('Knn(n_neighbors=11) ROC curve')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred_proba)

### Confusion Matrix
[Источник](https://medium.com/@djocz/confusion-matrix-aint-that-confusing-d29e18403327)

In [ ]:
#import confusion_matrix
from sklearn.metrics import confusion_matrix

y_pred = knn.predict(X_test)
confusion_matrix(y_test,y_pred)
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

### Precision
TP – True Positives
FP – False Positives
Precision – Accuracy of positive predictions.
Precision = TP/(TP + FP)

Вычисляя руками по формуле получаем такое же значение, что и методом класса "из коробки"

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred)

### Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

### Recall

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred)

### F-measure

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

### Дерево решений
[Источник](https://habr.com/ru/companies/ods/articles/322534/)
 Деревья решений используются в повседневной жизни в самых разных областях человеческой деятельности, порой и очень далеких от машинного обучения. Деревом решений можно назвать наглядную инструкцию, что делать в какой ситуации.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree = DecisionTreeClassifier(max_depth=5, random_state=42)

# Определим оптимальные параметры для дерева решений
tree_params = {"max_depth": range(1, 11), "max_features": range(4, 19)}
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(X_train,y_train)
tree_grid.best_params_, tree_grid.best_score_

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

print(f'Accuracy: {accuracy_score(y_test, tree_grid.predict(X_test))}')
print(f'Precision: {precision_score(y_test, tree_grid.predict(X_test))}')
print(f'Recall: {recall_score(y_test, tree_grid.predict(X_test))}')
print(f'F-measure: {f1_score(y_test, tree_grid.predict(X_test))}')
print(f'ROC AUC: {roc_auc_score(y_test, tree_grid.predict(X_test))}')